In [12]:
import torch
import transformer
import training
import importlib
importlib.reload(transformer);
importlib.reload(training);

In [17]:
#
# Hyper parameter
#
topic           = 'commits'
batch_size      = 64
max_iterations  = 2000
checkpoint_step = 1000
learning_rate   = 1e-4
eval_iters      = 200

#
# Network 
#
transformer.attention_heads_per_block = 8
transformer.attention_blocks          = 16
transformer.sample_size               = 32     # number of consecutive characters to predict from
transformer.embedding_size            = 256    # size of the embedding vectors
transformer.dropout                   = 0.2

# remember to set t.vocabulary_size when data is loaded

In [8]:
#
# Load and tokenize training text.
#
decoder, tokens = training.initializeTrainingData(topic)

In [14]:
#
# Load vocabulary and tokens
#
decoder, tokens = training.loadTrainingData(topic)

In [15]:
transformer.vocabulary_size = len(decoder)
training_data = training.createDataTensors(tokens)

In [16]:
#
# Model creation and validation
#
model = transformer.Transformer()
m = model.to(transformer.device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
start_iteration = 0

In [18]:
#
# Load model from checkpoint
#
start_iteration = 8000
checkpoint = torch.load(f'{topic}/{topic}-{start_iteration}.nn');
model.load_state_dict(checkpoint['model_state_dict']);
optimizer.load_state_dict(checkpoint['optimizer_state_dict']);

In [19]:
def checkpoint(step):
    train = model.training
    if train: model.eval();
    print(f"{step}: checkpoint...")
    losses = training.estimate_loss(model, training_data, eval_iters, transformer.sample_size, batch_size, transformer.device)
    print(f"{step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, f'{topic}/{topic}-{step}.nn');

    dummy_input = torch.randint(low=0, high=t.vocabulary_size, size=(1, t.sample_size), dtype=torch.long)
    torch.onnx.export(model, dummy_input, f"{topic}/{topic}-{step}.onnx");

    print(f"{step}: checkpoint saved.")
    if train: model.train();
     

In [21]:
#
# Training
#
model.train()
for iter in range(max_iterations):
    current_iteration = iter + start_iteration
    if current_iteration % checkpoint_step == 0 and (start_iteration == 0 or current_iteration > start_iteration):
        checkpoint(current_iteration)

    if current_iteration % 250 == 0:
        print(f"{current_iteration}: training")

    xb, yb = training.get_batch(training_data['train'], transformer.sample_size, batch_size, transformer.device)
    _, loss = model(xb,yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

current_iteration += 1
checkpoint(current_iteration)

8000: training
8250: training
8500: training
8750: training
9000: checkpoint...
9000: train loss 1.4990, val loss 2.5709
============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

9000: checkpoint saved.
9000: training


### Commit message generation training

#### Hyper and network parameters
| Parameter                 | Value 
| :--------                 | ----:
| tokenizer steps           | 300
| sample size               | 32
| embedding size            | 256
| batch size                | 64
| learning rate             | 1e-4
| attention heads per block | 8
| attention blocks          | 16
| dropout ratio             | 0.2


#### Training results
| Iteration | Loss (training)   | Loss (validation)
| :-------: | :-------------:   | :---------------:
| 0         | 6.2128            | 6.2222
| 1000      | 3.2123            | 3.3551
| 2000      | 3.2123            | 3.3551
| 3000      | 2.2692            | 2.6352
| 4000      | 2.0836            | 2.5631
| 5000      | 1.9396            | 2.5280
| 6000      | 1.8112            | 2.5055
| 7000      | 1.7079            | 2.5406
| 8000      | 1.5978            | 2.5683
| 9000      | 1.4990            | 2.5709

In [20]:
model.eval()
decode = lambda l: ''.join([decoder[i] for i in l])
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=t.device), max_tokens=500)[0].tolist()))

Änderung
XmlWriter DatabaseInfo und Architektur bei Condition von Auch bereits verarbeitet sollte Matching für IsReleaseWatcher refactored
Email von SystemInfos und ArgumentNullValue and implemented code issue
* Update radious old config via flow postal
- changed version to 1.1.0
zip to enjust type to für DeviceTypes (Toolbar wenn es unterstützt Wert
Hilfe implementiert  (Fehler und entfernt)
ServicesMsi - Checkbox der Mappings erweitert (um result zu esolled)
Single Erkennung von UPE-Number wird jetzt aufgetauscht
Bug bei Innerhalten von Hinträge es Zeiugriff wieder sich nicht mehr exportiert)
setting um ShortNameString() auf UKFD zur added fehlerhaft Fool
XOMD.net und FirmwareVersion das Info/AMP Dialog nicht angepasst
Reihenfolge: vlt nun die Datenbank zu 
Schema gefixt
Datenbankwänk fertig nun ausgelagert
Neue Adapter geprüft und web.config log vorschö
